In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 验证信息熵的计算
# 公式：H(x) = - sum( p(x) * log2( p(x) ) )
# ---------------------------------------------------------

# 1. 对应幻灯片里 "32支球队夺冠概率相同" 的情况
# log2(1/32) = -5
print("Case 1 (1/32):", np.log2(1/32))


# 2. 对应 "抛硬币" (50% vs 50%) - 不确定性最大
# 结果为 -1.0 (取绝对值后熵为 1)
print("Case 2 (1/2, 1/2):", 1/2 * np.log2(1/2) + 1/2 * np.log2(1/2))


# 3. 对应 "一个稍微确定的事件" (1/3 vs 2/3)
# 结果绝对值变小了，说明不确定性在降低
print("Case 3 (1/3, 2/3):", 1/3 * np.log2(1/3) + 2/3 * np.log2(2/3))


# 4. 对应 "极其确定的事件" (99% vs 1%) - 比如巴西队踢弱队
# 结果为 -0.08 (取绝对值后非常接近0)，说明几乎没有悬念
print("Case 4 (0.01, 0.99):", 0.01 * np.log2(0.01) + 0.99 * np.log2(0.99))

Case 1 (1/32): -5.0
Case 2 (1/2, 1/2): -1.0
Case 3 (1/3, 2/3): -0.9182958340544896
Case 4 (0.01, 0.99): -0.08079313589591118


In [3]:
import pandas as pd

# 路径解释：
# ".." 代表返回上一级目录 (从 day15 回到 ai-study)
# "/day14/data/titanic.txt" 进入目标文件夹找到文件
titan = pd.read_csv("../day14/data/titanic.txt")

# 查看数据是否读取成功
print(titan.head())

# 查看数据类型和缺失值情况
titan.info()

   row.names pclass  survived  \
0          1    1st         1   
1          2    1st         0   
2          3    1st         0   
3          4    1st         0   
4          5    1st         1   

                                              name      age     embarked  \
0                     Allen, Miss Elisabeth Walton  29.0000  Southampton   
1                      Allison, Miss Helen Loraine   2.0000  Southampton   
2              Allison, Mr Hudson Joshua Creighton  30.0000  Southampton   
3  Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)  25.0000  Southampton   
4                    Allison, Master Hudson Trevor   0.9167  Southampton   

                         home.dest room      ticket   boat     sex  
0                     St Louis, MO  B-5  24160 L221      2  female  
1  Montreal, PQ / Chesterville, ON  C26         NaN    NaN  female  
2  Montreal, PQ / Chesterville, ON  C26         NaN  (135)    male  
3  Montreal, PQ / Chesterville, ON  C26         NaN    NaN  female  

In [4]:
titan

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male
...,...,...,...,...,...,...,...,...,...,...,...
1308,1309,3rd,0,"Zakarian, Mr Artun",NaN,NaN,NaN,NaN,NaN,NaN,male
1309,1310,3rd,0,"Zakarian, Mr Maprieder",NaN,NaN,NaN,NaN,NaN,NaN,male
1310,1311,3rd,0,"Zenn, Mr Philip",NaN,NaN,NaN,NaN,NaN,NaN,male
1311,1312,3rd,0,"Zievens, Rene",NaN,NaN,NaN,NaN,NaN,NaN,female


In [5]:
# 1. 处理数据，找出特征值 (x) 和目标值 (y)
# 我们选择了 'pclass' (船舱等级), 'age' (年龄), 'sex' (性别) 作为判断依据
x = titan[['pclass', 'age', 'sex']]

y = titan['survived']

# 2. 检查是否有空值
# 这一步非常重要，因为决策树等算法通常无法直接处理空值 (NaN)
print(x.info())

# 3. 查看数据的统计描述
# include='all' 可以让我们看到非数字列 (如 sex) 的统计信息 (比如有多少种性别)
x.describe(include='all')

,pclass,age,sex
count,1313,633.000000,1313
unique,3,NaN,2
top,3rd,NaN,male
freq,711,NaN,850
mean,NaN,31.194181,NaN
std,NaN,14.747525,NaN
min,NaN,0.166700,NaN
25%,NaN,21.000000,NaN
50%,NaN,30.000000,NaN
75%,NaN,41.000000,NaN


In [6]:
# ---------------------------------------------------------
# 第一步：查看数据现状（可选）
# ---------------------------------------------------------
# 查看年龄的最大值，了解一下数据范围
print("最大年龄:", x.loc[:, 'age'].max())


# ---------------------------------------------------------
# 第二步：填补缺失值 (核心步骤)
# ---------------------------------------------------------
# 1. 计算 'age' 列的平均值
mean = x['age'].mean()
print(f"年龄均值: {mean}")

# 2. 使用均值填充缺失的部分
# 注意：这里使用 .loc[:, 'age'] 是为了避免 pandas 报 SettingWithCopyWarning 警告
# fillna(mean) 的意思就是：如果是空值，就填入 mean，如果不是空值，保持原样
x.loc[:, 'age'] = x.loc[:, 'age'].fillna(mean)


# ---------------------------------------------------------
# 第三步：验证结果
# ---------------------------------------------------------
# 再次查看信息，确认 'age' 列的 non-null count 是否已经等于总行数
print(x.info())

最大年龄: 71.0
年龄均值: 31.19418104265403
<class 'pandas.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1313 non-null   str    
 1   age     1313 non-null   float64
 2   sex     1313 non-null   str    
dtypes: float64(1), str(2)
memory usage: 30.9 KB
None


In [7]:
# 分割数据集到训练集和测试集
# test_size=0.25 表示 25% 的数据用来做测试，75% 用来训练
# random_state=4 固定随机种子，保证每次切分的结果都一样
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)

# 查看训练集前几行数据
print(x_train.head())

# 验证数据类型
print(type(x_train))

# 查看训练集中目标值（存活人数）的总和
# 因为存活是1，死亡是0，sum的结果就是存活的总人数
print(sum(y_train))

    pclass        age     sex
598    2nd  30.000000    male
246    1st  62.000000    male
905    3rd  31.194181  female
300    1st  31.194181  female
509    2nd  64.000000    male
<class 'pandas.DataFrame'>
334


In [8]:
# 1. 创建一个临时表格 z，把特征值和目标值拼在一起
# 这样方便我们同时筛选“性别”和查看“存活状态”
z = x_train.copy()
z['survived'] = y_train

# 2. 查看【女性】的存活情况
print("--- 女性存活情况 (1=存活, 0=死亡) ---")
print(z[z['sex'] == 'female']['survived'].value_counts())

# 3. 查看【男性】的存活情况 (这是补充的部分)
print("\n--- 男性存活情况 (1=存活, 0=死亡) ---")
print(z[z['sex'] == 'male']['survived'].value_counts())

--- 女性存活情况 (1=存活, 0=死亡) ---
survived
1    230
0    111
Name: count, dtype: int64

--- 男性存活情况 (1=存活, 0=死亡) ---
survived
0    539
1    104
Name: count, dtype: int64


In [9]:
x_train.to_dict(orient='records')

[{'pclass': '2nd', 'age': 30.0, 'sex': 'male'},
 {'pclass': '1st', 'age': 62.0, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '1st', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '2nd', 'age': 64.0, 'sex': 'male'},
 {'pclass': '1st', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '3rd', 'age': 24.0, 'sex': 'female'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '2nd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 21.0, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '2nd', 'age': 23.0, 'sex': 'female'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '1st', 'age': 4

In [10]:
from sklearn.feature_extraction import DictVectorizer

# 1. 实例化转换器
# sparse=False 表示返回一个完整的数组(ndarray)，而不是稀疏矩阵
# 这样打印出来看的时候更直观，看到的是实实在在的矩阵
dict_vec = DictVectorizer(sparse=False)

# 2. 对训练集进行特征抽取 (fit_transform)
# 这一步非常关键：
# x_train.to_dict(orient="records") 把 DataFrame 转换成字典列表
# fit_transform 会自动学习有哪些类别，并转换成数字矩阵
x_train = dict_vec.fit_transform(x_train.to_dict(orient="records"))

# 3. 查看转换后的数据类型 (应该是 numpy.ndarray)
print(type(x_train))

# 4. 查看特征名字
# 这能让你看到原来的 'sex' 变成了 'sex=female' 和 'sex=male'
print(dict_vec.get_feature_names_out())

print('-' * 50)

# 5. 对测试集进行特征抽取 (transform)
# 注意：测试集只能用 transform，不能用 fit！
# 我们要用训练集学到的规则来转换测试集
x_test = dict_vec.transform(x_test.to_dict(orient="records"))

# 6. 打印看看长什么样
print(x_train)

<class 'numpy.ndarray'>
['age' 'pclass=1st' 'pclass=2nd' 'pclass=3rd' 'sex=female' 'sex=male']
--------------------------------------------------
[[30.          0.          1.          0.          0.          1.        ]
 [62.          1.          0.          0.          0.          1.        ]
 [31.19418104  0.          0.          1.          1.          0.        ]
 ...
 [34.          0.          1.          0.          0.          1.        ]
 [46.          1.          0.          0.          0.          1.        ]
 [31.19418104  0.          0.          1.          0.          1.        ]]


In [11]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

# 1. 初始化决策树分类器
# 默认 criterion='gini' (基尼系数)，你可以改成 'entropy' (信息增益)
# max_depth=None (默认深度不限)，如果树太深容易过拟合，可以设置一个数字限制它
dec = DecisionTreeClassifier(criterion='entropy')

# 2. 训练模型 (Fit)
# 这一步，决策树开始疯狂计算每个特征的熵和信息增益，构建树的结构
dec.fit(x_train, y_train)

# 3. 预测并评估准确率
# score 内部会先 predict(x_test)，然后和 y_test 比较计算准确率
print("预测的准确率: ", dec.score(x_test, y_test))

# 4. 导出决策树的结构 (可视化)
# out_file: 导出的文件名，会在当前目录下生成一个 tree.dot 文件
# feature_names: 把很难懂的 X[0], X[1] 替换成 'age', 'sex=male' 这样的人话
# 💡 优化点：直接用 dict_vec.get_feature_names_out() 自动获取，不用手动去敲列表
export_graphviz(dec, out_file="./tree.dot",
                feature_names=dict_vec.get_feature_names_out())

 0.8085106382978723


参数调整，下面是预剪枝

In [12]:
# 1. 数据集划分
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)

# 2. 特征工程 (字典特征抽取)
dict = DictVectorizer(sparse=False)
x_train = dict.fit_transform(x_train.to_dict(orient="records"))
x_test = dict.transform(x_test.to_dict(orient="records"))

# 3. 决策树模型训练 (带有剪枝参数)
# 修改max_depth为10, 发现提升了, min_impurity_decrease带来的增益要大于0.01才会进行划分
dec = DecisionTreeClassifier(max_depth=7, min_impurity_decrease=0.01, min_samples_split=20)
dec.fit(x_train, y_train)

# 4. 预测准确率
print("预测的准确率：", dec.score(x_test, y_test))

# 5. 导出决策树结构
export_graphviz(dec, out_file="tree1.dot",
                feature_names=dict.get_feature_names_out())

预测的准确率： 0.8206686930091185


控制深度是为了防止过拟合

随机森林

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# 随机森林进行预测 （超参数调优），n_jobs充分利用多核的一个参数
rf = RandomForestClassifier(n_jobs=-1)

# 120, 200, 300, 500, 800, 1200,n_estimators森林中决策树的数目，也就是分类器的数目
# max_samples 是最大样本数
# bagging类型
param = {"n_estimators": [1500, 2000, 5000], "max_depth": [2, 3, 5, 8, 15, 25]}

# 网格搜索与交叉验证
gc = GridSearchCV(rf, param_grid=param, cv=3)

gc.fit(x_train, y_train)

print("准确率：", gc.score(x_test, y_test))

print("查看选择的参数模型：", gc.best_params_)

print("选择最好的模型是：", gc.best_estimator_)

# print("每个超参数每次交叉验证的结果：", gc.cv_results_)

准确率： 0.8328267477203647
查看选择的参数模型： {'max_depth': 3, 'n_estimators': 1500}
选择最好的模型是： RandomForestClassifier(max_depth=3, n_estimators=1500, n_jobs=-1)
